In [1]:
import pandas as pd 
import numpy as np 
from decimal import *
from sklearn.preprocessing import MinMaxScaler

import os
import sys
sys.path.append('/Users/maba4574/Desktop/Work/Projects/MIMIC/patient_clustering_EHR/helper_functions')
import helpers 


import warnings
warnings.filterwarnings("ignore")



2023-10-31 21:18:02.688792: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
## preprocess --> blood-gas, blood-gas-arterial, labs, urine-output, vitals, vasopressors

folder_sepsis = '../sepsis3_queries/data'
sepsis3 = pd.read_csv(f'{folder_sepsis}/sepsis-df-3.csv')
eq = pd.read_csv(f'{folder_sepsis}/elixhauser_quan.csv')

folder = 'data_stage_1'
static = pd.read_csv(f'{folder}/static.csv')
time_series = pd.read_csv(f'{folder}/time_series.csv')

### Extract the angus cohort

In [4]:
# sepsis3 = sepsis3[sepsis3.sepsis_angus == 1]
# time_series = time_series[time_series.icustay_id.isin(sepsis3.icustay_id)]
# static = static[static.icustay_id.isin(sepsis3.icustay_id)]

### match the EQ hadm_id to sepsis3 icustastay_id

In [5]:
eq = eq[eq.hadm_id.isin(sepsis3.hadm_id)]
eq = pd.merge(sepsis3[['icustay_id', 'hadm_id']], eq, on='hadm_id', how='inner').drop('hadm_id',axis=1)


In [6]:
eq.head()

,icustay_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,200061,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,200075,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,200087,1.0,1.0,0.0,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,200116,1.0,0.0,0.0,1.0,0.0,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,200131,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Create 1 hour intervals for time series

In [ ]:
### 1 hours intervals

KEYS = ['icustay_id']

time_series[['charttime']] = time_series[['charttime']].apply(pd.to_datetime)

sepsis3[["intime", "outtime"]] = sepsis3[["intime", "outtime"]].apply(pd.to_datetime)

print("Hourly buckets")
#### hourly buckets ####################################
to_hours = lambda x: max(0, x.days*24 + x.seconds // 3600)
#  the value is str, convert to numeric
#time_series['value'] = pd.to_numeric(time_series['value'], 'coerce')
# join and add in labs_vital the icu intime and outime, to separate in hourly buckets
sepsis3 = sepsis3.set_index(KEYS)
time_series = time_series.set_index(KEYS).join(sepsis3[['intime', 'outtime']])
# to hourly buckets
time_series['hours_in'] = (time_series['charttime'] - time_series['intime']).apply(to_hours)

In [ ]:
time_series  = time_series.drop(['charttime', 'intime', 'outtime'], axis=1)

In [ ]:
time_series.reset_index(inplace=True)

In [ ]:
aggregated_df = time_series.groupby(['icustay_id', 'hours_in']).agg(lambda x: x.dropna().tolist())
aggregated_df

In [ ]:
aggregated_df = aggregated_df.applymap(lambda x: x[0] if x else np.nan)
aggregated_df

In [ ]:

# Group the DataFrame by "icustay_id"
grouped = aggregated_df.groupby("icustay_id")

# Apply the function to each patient group
result_df = grouped.apply(helpers.fill_missing_hours).reset_index(drop=False)


result_df

In [ ]:
time_series = result_df.drop('level_1', axis=1)

### Categorize age, ethnicity, gender

In [7]:
sepsis3.columns

Index(['icustay_id', 'hadm_id', 'excluded', 'intime', 'outtime', 'dbsource',
       'suspected_infection_time_poe', 'suspected_infection_time_poe_days',
       'specimen_poe', 'positiveculture_poe', 'antibiotic_time_poe',
       'blood_culture_time', 'blood_culture_positive', 'age', 'gender',
       'is_male', 'ethnicity', 'race_white', 'race_black', 'race_hispanic',
       'race_other', 'metastatic_cancer', 'diabetes', 'height', 'weight',
       'bmi', 'first_service', 'hospital_expire_flag', 'thirtyday_expire_flag',
       'icu_los', 'hosp_los', 'sepsis_angus', 'sepsis_martin',
       'sepsis_explicit', 'septic_shock_explicit', 'severe_sepsis_explicit',
       'sepsis_nqf', 'sepsis_cdc', 'sepsis_cdc_simple', 'elixhauser_hospital',
       'vent', 'sofa', 'lods', 'sirs', 'qsofa', 'qsofa_sysbp_score',
       'qsofa_gcs_score', 'qsofa_resprate_score', 'composite_outcome',
       'blood culture', 'suspicion_poe', 'abx_poe', 'sepsis-3', 'sofa>=2'],
      dtype='object')

In [8]:
eq.columns

Index(['icustay_id', 'congestive_heart_failure', 'cardiac_arrhythmias',
       'valvular_disease', 'pulmonary_circulation', 'peripheral_vascular',
       'hypertension', 'paralysis', 'other_neurological', 'chronic_pulmonary',
       'diabetes_uncomplicated', 'diabetes_complicated', 'hypothyroidism',
       'renal_failure', 'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression'],
      dtype='object')

In [9]:
# ### One hot encode static data and bin age

ethnicity = sepsis3.reset_index()[['race_white', 'race_black', 'race_hispanic',
       'race_other']]

static['age_bucket'] = static['age'].apply(helpers.categorize_age)

#one_hot_ethinicity_grouped = pd.get_dummies(static['ethnicity'])
one_hot_gender = pd.get_dummies(static['gender'])
one_hot_age = pd.get_dummies(static['age_bucket'])
one_hot_gcs = pd.get_dummies(static['response_scale'])

static.drop(['ethnicity', 'gender', 'age', 'age_bucket', 'response_scale'], axis=1, inplace=True)
static = static.join(one_hot_gender).join(one_hot_age).join(ethnicity).join(one_hot_gcs)




### Missing values

In [10]:
static

,icustay_id,hadm_id,vent,subject_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,...,adults,seniors,youth,race_white,race_black,race_hispanic,race_other,gcs_13_15,gcs_9_12,gcs_<8
0,200061,121149,0,74282,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
1,200075,132255,1,67800,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
2,200087,124231,1,61871,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,200116,164386,0,61654,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
4,200131,187834,1,47250,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779,299930,175890,0,47967,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
5780,299948,183226,0,66696,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
5781,299949,199962,1,53283,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
5782,299950,110990,0,98649,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0


In [11]:
#decide if you want the eq or the ahrq comorbidities
## currently the static data have the ahrq, so i will drop them and replace them with the eq comorbidities becuase it's less empty
demographics = static[['icustay_id', 'hadm_id', 'vent', 'subject_id','F', 'M', 'adults', 'seniors', 'youth',
       'race_white', 'race_black', 'race_hispanic', 'race_other', 'gcs_13_15',
       'gcs_9_12', 'gcs_<8']]
demographics

static = pd.merge(demographics, eq, on='icustay_id', how='inner').drop('hadm_id',axis=1).set_index('icustay_id')
static


,vent,subject_id,F,M,adults,seniors,youth,race_white,race_black,race_hispanic,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
icustay_id,,,,,,,,,,,,,,,,,,,,,
200061,0,74282,0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
200075,1,67800,1,0,0,1,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
200087,1,61871,0,1,0,1,0,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200116,0,61654,0,1,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200131,1,47250,1,0,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299930,0,47967,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299948,0,66696,1,0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299949,1,53283,0,1,0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:

static = static.loc[:, ((static == 0).sum(axis=0)/len(static)) != 1]

#static = static.loc[:, ((static == 0).sum(axis=0)/len(static)) != 1]

for column_name in static.columns:
    column = static[column_name]
    # Get the count of Zeros in column 
    count = (column == 0).sum()
    percentage = (count / len(static))* 100 
    print('% of zeros in column ', column_name, ' is : ', percentage, ' count', count, 'out of ', len(static) )


% of zeros in column  vent  is :  51.901798063623794  count 3002 out of  5784
% of zeros in column  subject_id  is :  0.0  count 0 out of  5784
% of zeros in column  F  is :  55.6881051175657  count 3221 out of  5784
% of zeros in column  M  is :  44.3118948824343  count 2563 out of  5784
% of zeros in column  adults  is :  58.90387275242047  count 3407 out of  5784
% of zeros in column  seniors  is :  43.75864453665284  count 2531 out of  5784
% of zeros in column  youth  is :  97.3374827109267  count 5630 out of  5784
% of zeros in column  race_white  is :  27.351313969571233  count 1582 out of  5784
% of zeros in column  race_black  is :  91.30359612724757  count 5281 out of  5784
% of zeros in column  race_hispanic  is :  96.74965421853389  count 5596 out of  5784
% of zeros in column  race_other  is :  84.5954356846473  count 4893 out of  5784
% of zeros in column  gcs_13_15  is :  44.69225449515906  count 2585 out of  5784
% of zeros in column  gcs_9_12  is :  83.64453665283541  

In [13]:
# create a Boolean mask for the rows to remove
# mask = time_series['hours_in'] == 24

# # select all rows except the ones that contain 'Coca Cola'
# time_series = time_series[~mask]

# # print the resulting DataFrame
# time_series[time_series.hours_in == 24]

In [ ]:
time_series = time_series.drop(['specimen'], axis=1)

temp = time_series.groupby('icustay_id').mean()
for column_name in temp.columns:
    column = temp[column_name]
    # Get the count of Zeros in column 
    count = (column.isna()).sum()
    percentage = (count / len(temp)) * 100 
    print(column_name, ' is : ', percentage, ' % empty' )


## Group the comorbidities to upper hierarchy codes


congestive_heart_failure: --circ
cardiac_arrhythmias: circ
pulmonary_circulation: circ

calvular_disease: circ-circinfect-circcongenital

peripheral_vascular: circ-circinfect-circcongenital

hypertension: circ-hypertension
chronic_pulmonary: circ-resp
renal_failure: circ-renal
liver_disease: circ-liver

paralysis: nervousandsense
other_neurological: nervousandsense

diabetes_uncomplicated: encocrinalnutrional
diabetes_complicated: encocrinalnutrional
hypothyroidism: encocrinalnutrional
obesity: encocrinalnutrional
weight_loss: encocrinalnutrional
fluid_electrolyte: encocrinalnutrional

rheumatoid_arthritis: musculoskeletal

peptic_ulcer: digestive


aids: infectiousparasitic

lymphoma: neoplasms
metastatic_cancer: neoplasms
solid_tumor: neoplasms

coagulopathy: bloodandbloodforming
blood_loss_anemia: bloodandbloodforming
deficiency_anemias: bloodandbloodforming

psychoses: mental
depression: mental

alcohol_abuse
drug_abuse




In [14]:
static.columns

Index(['vent', 'subject_id', 'F', 'M', 'adults', 'seniors', 'youth',
       'race_white', 'race_black', 'race_hispanic', 'race_other', 'gcs_13_15',
       'gcs_9_12', 'gcs_<8', 'congestive_heart_failure', 'cardiac_arrhythmias',
       'valvular_disease', 'pulmonary_circulation', 'peripheral_vascular',
       'hypertension', 'paralysis', 'other_neurological', 'chronic_pulmonary',
       'diabetes_uncomplicated', 'diabetes_complicated', 'hypothyroidism',
       'renal_failure', 'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression'],
      dtype='object')

In [15]:
diabetes = pd.DataFrame(static[["diabetes_complicated", "diabetes_uncomplicated"]].max(axis=1), columns=['diabetes'])

circulatory = pd.DataFrame(static[["congestive_heart_failure", "cardiac_arrhythmias", "pulmonary_circulation"]].max(axis=1), columns=['circulatory'])
circulatory_infectious_congenital = pd.DataFrame(static[["valvular_disease", "peripheral_vascular"]].max(axis=1), columns=['circulatory_infectious_congenital'])
nervousandsense = pd.DataFrame(static[[ "other_neurological","paralysis"]].max(axis=1), columns=['nervous_and_sense'])
#
encocrinalnutrional = pd.DataFrame(static[["diabetes_uncomplicated", "diabetes_complicated", "hypothyroidism", "obesity", "weight_loss", "fluid_electrolyte"]].max(axis=1), columns=['endocrinal_nutritional'])
mental = pd.DataFrame(static[["psychoses", 'depression']].max(axis=1), columns=['mental'])
neoplasms = pd.DataFrame(static[["lymphoma", "metastatic_cancer", "solid_tumor"]].max(axis=1), columns=['neoplasms'])
bloodandbloodforming = pd.DataFrame(static[["coagulopathy", "blood_loss_anemia", "deficiency_anemias"]].max(axis=1), columns=['blood_and_blood_forming'])
#

static = static.join(diabetes).join(circulatory).join(circulatory_infectious_congenital).join((nervousandsense)).join(encocrinalnutrional).join(neoplasms).join(bloodandbloodforming).join(mental)
#
static

,vent,subject_id,F,M,adults,seniors,youth,race_white,race_black,race_hispanic,...,psychoses,depression,diabetes,circulatory,circulatory_infectious_congenital,nervous_and_sense,endocrinal_nutritional,neoplasms,blood_and_blood_forming,mental
icustay_id,,,,,,,,,,,,,,,,,,,,,
200061,0,74282,0,1,1,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
200075,1,67800,1,0,0,1,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
200087,1,61871,0,1,0,1,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
200116,0,61654,0,1,1,0,0,1,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
200131,1,47250,1,0,0,1,0,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299930,0,47967,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299948,0,66696,1,0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299949,1,53283,0,1,0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
static = static.loc[:, ((static == 0).sum(axis=0)/len(static)) != 1]

for column_name in static.columns:
    column = static[column_name]
    # Get the count of Zeros in column 
    count = (column == 0).sum()
    percentage = (count / len(static))* 100 
    print('% of zeros in column ', column_name, ' is : ', percentage, ' count', count, 'out of ', len(static) )

% of zeros in column  vent  is :  51.901798063623794  count 3002 out of  5784
% of zeros in column  subject_id  is :  0.0  count 0 out of  5784
% of zeros in column  F  is :  55.6881051175657  count 3221 out of  5784
% of zeros in column  M  is :  44.3118948824343  count 2563 out of  5784
% of zeros in column  adults  is :  58.90387275242047  count 3407 out of  5784
% of zeros in column  seniors  is :  43.75864453665284  count 2531 out of  5784
% of zeros in column  youth  is :  97.3374827109267  count 5630 out of  5784
% of zeros in column  race_white  is :  27.351313969571233  count 1582 out of  5784
% of zeros in column  race_black  is :  91.30359612724757  count 5281 out of  5784
% of zeros in column  race_hispanic  is :  96.74965421853389  count 5596 out of  5784
% of zeros in column  race_other  is :  84.5954356846473  count 4893 out of  5784
% of zeros in column  gcs_13_15  is :  44.69225449515906  count 2585 out of  5784
% of zeros in column  gcs_9_12  is :  83.64453665283541  

In [17]:
#time_series = time_series.drop(['charttime', 'intime', 'outtime'], axis=1)

In [18]:
static = static.drop(['subject_id'], axis=1)
static.columns

Index(['vent', 'F', 'M', 'adults', 'seniors', 'youth', 'race_white',
       'race_black', 'race_hispanic', 'race_other', 'gcs_13_15', 'gcs_9_12',
       'gcs_<8', 'congestive_heart_failure', 'cardiac_arrhythmias',
       'valvular_disease', 'pulmonary_circulation', 'peripheral_vascular',
       'hypertension', 'paralysis', 'other_neurological', 'chronic_pulmonary',
       'diabetes_uncomplicated', 'diabetes_complicated', 'hypothyroidism',
       'renal_failure', 'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression', 'diabetes', 'circulatory',
       'circulatory_infectious_congenital', 'nervous_and_sense',
       'endocrinal_nutritional', 'neoplasms', 'blood_and_blood_forming',
       'mental'],
      dtype='object')

### Drop missing values 

In [19]:

print('static shape before dropping empty values', static.shape)

static_ = static.loc[:, static.gt(0).mean() >= .001]
print('static shape after dropping empty values', static_.shape)

#where number of nonzeros is less than 1% of length of columns
for column_name in static_.columns:
    column = static_[column_name]
    # Get the count of Zeros in column 
    count = (column == 1).sum()
    percentage = (count / len(static_))* 100 
    print(column_name, ' is : ', percentage, ' % present' )

static shape before dropping empty values (5784, 51)
static shape after dropping empty values (5784, 51)
vent  is :  48.09820193637621  % present
F  is :  44.3118948824343  % present
M  is :  55.6881051175657  % present
adults  is :  41.09612724757953  % present
seniors  is :  56.24135546334716  % present
youth  is :  2.6625172890733055  % present
race_white  is :  72.64868603042876  % present
race_black  is :  8.69640387275242  % present
race_hispanic  is :  3.250345781466114  % present
race_other  is :  15.404564315352697  % present
gcs_13_15  is :  55.307745504840945  % present
gcs_9_12  is :  16.35546334716459  % present
gcs_<8  is :  28.336791147994468  % present
congestive_heart_failure  is :  25.069156293222683  % present
cardiac_arrhythmias  is :  36.72199170124482  % present
valvular_disease  is :  9.232365145228217  % present
pulmonary_circulation  is :  8.108575380359612  % present
peripheral_vascular  is :  8.07399723374827  % present
hypertension  is :  58.437067773167364 

In [ ]:
temp = time_series.groupby('icustay_id').mean()

ts_to_delete = []
for column_name in temp.columns:
    #print(column_name)
    column = temp[column_name]
    # Get the count of Zeros in column 
    count = (column.isna()).sum()

    percentage = (count / len(temp)) * 100 
    if percentage>99:
        print(column_name)
        ts_to_delete.append(column_name)
    print(column_name, ' is : ', percentage, ' % empty' )


print('time series shape before dropping empty values', time_series.shape)        
time_series_ = time_series.loc[:, ~time_series.columns.isin(ts_to_delete)]
print('time series shape after dropping empty values', time_series_.shape)  
# this attribure contains text values



### Time series imputation

In [ ]:
ID_COLS = ['icustay_id']
HOUR_COLS = ['hours_in']
## aggregation of time series using mean value: representation <ts2>
time_series_2d = time_series_.copy()
time_series_2d = time_series_2d.groupby(ID_COLS).mean()
#time_series_agg = time_series_agg.fillna(-1)

time_series_ = time_series_.set_index(ID_COLS)
time_series_ = helpers.simple_imputer(time_series_, ID_COLS)
print("X after imputation",len(time_series_.index.unique()))

In [ ]:
time_series_

### Save the non-standardized datasets

In [ ]:
if not os.path.isdir(f"representations"):

    os.makedirs(f"representations")

# sepsis 3
# time_series_2d.to_csv('representations/time_series_2d.csv')
# time_series_.to_csv('representations/time_series_3d.csv')

# angus
# time_series_2d.to_csv('representations/time_series_2d_angus.csv')
# time_series_.to_csv('representations/time_series_3d_angus.csv')

### Standardize time-series and create 2d and 3d arrays

In [ ]:

time_series_ = time_series_.reset_index()
time_series_.set_index(ID_COLS + HOUR_COLS, inplace=True)

time_series_3d = time_series_.values.reshape(int(time_series_.shape[0]/24),24,time_series_.shape[1])



In [ ]:
# standardize time series

scaler = MinMaxScaler(feature_range=(0,1))
time_series_3d_scaled = scaler.fit_transform(time_series_3d.reshape(-1, time_series_3d.shape[-1])).reshape(time_series_3d.shape)
# fill nan with -1
time_series_3d_scaled = np.nan_to_num(time_series_3d_scaled, nan=-1)

time_series_2d_scaled = scaler.fit_transform(time_series_2d)
# fill nan with -1
time_series_2d_scaled = np.nan_to_num(time_series_2d_scaled, nan=-1)


time_series_2d_scaled_df = pd.DataFrame(time_series_2d_scaled, columns=time_series_2d.columns, index=time_series_2d.index )
time_series_2d_scaled_df = time_series_2d_scaled_df.drop(['hours_in'], axis=1)
multimodal_2d = time_series_2d_scaled_df.join(static)


In [ ]:
time_series_3d_scaled.shape

In [ ]:
time_series_2d_scaled.shape

In [ ]:
time_series_2d_scaled_df.shape

In [ ]:
time_series_2d_scaled_df.columns

In [20]:
static

,vent,F,M,adults,seniors,youth,race_white,race_black,race_hispanic,race_other,...,psychoses,depression,diabetes,circulatory,circulatory_infectious_congenital,nervous_and_sense,endocrinal_nutritional,neoplasms,blood_and_blood_forming,mental
icustay_id,,,,,,,,,,,,,,,,,,,,,
200061,0,0,1,1,0,0,0,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
200075,1,1,0,0,1,0,1,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
200087,1,0,1,0,1,0,1,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
200116,0,0,1,1,0,0,1,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
200131,1,1,0,0,1,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299930,0,1,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299948,0,1,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299949,1,0,1,0,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# sepsis3
# np.save('representations/time_series_3d_scaled.npy', time_series_3d_scaled) # save
# np.save('representations/time_series_2d_scaled.npy', time_series_3d_scaled) # save


# time_series_2d_scaled_df.to_csv('representations/time_series_2d_scaled_df.csv')
# static_.to_csv('representations/static_eq.csv')
# multimodal_2d.to_csv('representations/multimodal_2d.csv')



# angus
# np.save('representations/time_series_3d_scaled_angus.npy', time_series_3d_scaled) # save
# np.save('representations/time_series_2d_scaled_angus.npy', time_series_3d_scaled) # save


# time_series_2d_scaled_df.to_csv('representations/time_series_2d_scaled_df_angus.csv')
#static_.to_csv('representations/static_eq_angus.csv')
# multimodal_2d.to_csv('representations/multimodal_2d_angus.csv')
